### Código para obtener los centros de distribución de Soriana, Chedraui y Walmart

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import googlemaps
import folium
import os
import zipfile
from tqdm import tqdm
import geopandas as gpd
from selenium import webdriver
import time
from branca.element import Template, MacroElement
import re

#### Función general

In [4]:
def get_html(url, encoding=None, verify=False):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, verify=verify)
    if encoding is not None:
        response.encoding = encoding
    return BeautifulSoup(response.text, "html.parser")


##### Soriana

In [6]:
soriana=get_html(url["Soriana"], "utf-8")

C:\Users\claud\miniconda3\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.organizacionsoriana.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
#Obtener todo lo de la tabla
tabla=soriana.find_all("table", {"class": "table table-striped"})


In [8]:
all_tables_data = []  # Lista vacía

for table in tabla:  # Iterar sobre cada tabla
    headers = [th.get_text().strip() for th in table.find_all('th')]  # Obtener headers
    table_data = []  # Lista vacía para almacenar los datos de la tabla
    rows = table.find_all('tr')  # Obtener todas las filas de la tabla
    
    for row in rows:  # iterar sobre cada fila
        cells = row.find_all('td')  # Obtener las celdas de la fila
        if cells:  # Si hay celdas en la fila
            row_data = [cell.get_text().strip() for cell in cells]  
            if row_data:  # Si hay datos en la fila
                table_data.append(dict(zip(headers, row_data)))  # Agregar los datos de la fila a la lista de datos de la tabla

    if table_data:  # Si hay datos en la tabla
        all_tables_data.append(table_data)  # Agregar los datos de la tabla a la lista de datos de todas las tablas

In [9]:
#Pasar la lista a un dataframe
soriana_df=pd.DataFrame()
for i in range(len(all_tables_data)):
    soriana_df=pd.concat([soriana_df, pd.DataFrame(all_tables_data[i])], ignore_index=True)
# Reordenar la df
df_reorganized = pd.DataFrame(columns=["nombre", "direccion", "tipo"])

rename_map = {
    "CENTROS DE DISTRIBUCIÓN FRESCOS": "Fresco",
    "CENTROS DE DISTRIBUCIÓN SECOS": "Seco",
    "CENTRO DE DISTRIBUCIÓN  E-COMMERCE": "E-Commerce"
}


for _, row in soriana_df.iterrows():
    
    for col_name, tipo in rename_map.items():
        if pd.notna(row[col_name]):
            df_reorganized =pd.concat([df_reorganized, pd.DataFrame([[row[col_name], row["DIRECCIÓN"], tipo]], columns=["nombre", "direccion", "tipo"])], ignore_index=True)
soriana_df=df_reorganized
#Añadir tienda
soriana_df["tienda"]="Soriana"
soriana_df

,nombre,direccion,tipo,tienda
0,CDF Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Fresco,Soriana
1,CDF Cuautitlán,"Autopista México-Querétaro km 36.8,C.P. 54700 ...",Fresco,Soriana
2,CDF Villahermosa,"Carretera Villahermosa-Cárdenas Km 164.1,Ranch...",Fresco,Soriana
3,CDF Guadalajara,"Periférico Sur Manuel Gómez Morín No. 5890,Col...",Fresco,Soriana
4,CDF Tijuana,Prolongación Club de Leones 10535.Fraccionamie...,Fresco,Soriana
5,CDF Hermosillo,Av. Chetumal entre Guatemala y Belice No. 1121...,Fresco,Soriana
6,CDS Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Seco,Soriana
7,CDS Tultitlán,"Km. 14.5 Carretera Puente de Vigas Cuautitlán,...",Seco,Soriana
8,CDS Querétaro,"Autopista México-Querétaro Km 186 S/N, Col Ent...",Seco,Soriana
9,CDS Guadalajara,"Periférico Sur Manuel Gómez Morín #5890, Col. ...",Seco,Soriana


##### Chedraui

In [10]:
chedraui=get_html(url["Chedraui"],encoding="utf-8",verify=False)

C:\Users\claud\miniconda3\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.grupochedraui.com.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [11]:
#Obtener resultados
results=chedraui.find_all("div", {"class": "col-lg-12"})

#Obtener los nombres de los centros de distribución
nombres=chedraui.find_all("h2", {"class": "font-bold font-color-naranja font-size-18"})
direcciones=chedraui.find_all("ul", {"class": "ventajas-ul"})
#Obtener las direcciones
direcciones=[direccion.get_text().strip() for direccion in direcciones]
#Eliminar los saltos de línea y los espacios
direcciones=[direccion.replace("\n", "").replace("  ", "").replace("\t", " ") for direccion in direcciones]

#Pasar a un dataframe
chedraui_df=pd.DataFrame({"nombre": [nombre.get_text().strip() for nombre in nombres],
                          "direccion": direcciones})

chedraui_df["tienda"]="Chedraui"
chedraui_df

,nombre,direccion,tienda
0,Centro de Distribución Cancún,"Carr. Cancún - Puerto Morelos, Km. 329.5 Cancú...",Chedraui
1,Centro de Distribución Guadalajara,"Carretera El Verde, El Castillo No. 2000, Col....",Chedraui
2,Centro de Distribución La Paz,"Colima S/N, Col. Pueblo Nuevo, La Paz Baja Cal...",Chedraui
3,Centro de Distribución Monterrey,"Jaime Nuno No. 3660, Col. Del Norte, Monterrey...",Chedraui
4,Centro de Distribución Teoloyucan,"Carretera Cuautitlán-Zumpango Lt. 28 Mz. 825, ...",Chedraui
5,Centro de Distribución Veracruz,"Carretera Veracruz-Cardel, Calle tres zapotes ...",Chedraui
6,Centro de Distribución Villahermosa,"Carretera Villahermosa-Macuspana Km. 12.5, Cor...",Chedraui
7,Centro de Distribución Tampico,"Av. Hidalgo No. 501, Col. Del Pueblo, Tampico,...",Chedraui


In [12]:
driver = webdriver.Chrome()
driver.get(url["Walmart"])
time.sleep(5)
html=driver.page_source
driver.quit()

In [13]:
walmart=BeautifulSoup(html, "html.parser")

In [14]:
results=walmart.find_all("a", {"class": "hfpxzc"})
results

[<a aria-label="Cedis eCommerce Walmart" class="hfpxzc" href="https://www.google.com.mx/maps/place/Cedis+eCommerce+Walmart/data=!4m7!3m6!1s0x85d2214c2617819f:0x2ac0d663c0d44ef6!8m2!3d19.7467676!4d-99.204067!16s%2Fg%2F11gtynlbtp!19sChIJn4EXJkwh0oUR9k7UwGPWwCo?authuser=0&amp;hl=es-419&amp;rclk=1" jsaction="pane.wfvdle5;focus:pane.wfvdle5;blur:pane.wfvdle5;auxclick:pane.wfvdle5;keydown:pane.wfvdle5;clickmod:pane.wfvdle5" jslog="12690;track:click,contextmenu;mutable:true;metadata:WyIwYWhVS0V3aVJyWTJpOFBtR0F4Vk82Y2tESGRJUEE2WVE4QmNJQlNnQSIsbnVsbCwxXQ=="></a>,
 <a aria-label="Cedis Walmart ecommerce" class="hfpxzc" href="https://www.google.com.mx/maps/place/Cedis+Walmart+ecommerce/data=!4m7!3m6!1s0x842f49df9293e1ff:0xc5cc3c098d25ee6b!8m2!3d20.4796628!4d-103.2616698!16s%2Fg%2F11rcndfsth!19sChIJ_-GTkt9JL4QRa-4ljQk8zMU?authuser=0&amp;hl=es-419&amp;rclk=1" jsaction="pane.wfvdle8;focus:pane.wfvdle8;blur:pane.wfvdle8;auxclick:pane.wfvdle8;keydown:pane.wfvdle8;clickmod:pane.wfvdle8" jslog="12690;tr

In [15]:
sopa=BeautifulSoup(str(results), "html.parser")
sopa

[<a aria-label="Cedis eCommerce Walmart" class="hfpxzc" href="https://www.google.com.mx/maps/place/Cedis+eCommerce+Walmart/data=!4m7!3m6!1s0x85d2214c2617819f:0x2ac0d663c0d44ef6!8m2!3d19.7467676!4d-99.204067!16s%2Fg%2F11gtynlbtp!19sChIJn4EXJkwh0oUR9k7UwGPWwCo?authuser=0&amp;hl=es-419&amp;rclk=1" jsaction="pane.wfvdle5;focus:pane.wfvdle5;blur:pane.wfvdle5;auxclick:pane.wfvdle5;keydown:pane.wfvdle5;clickmod:pane.wfvdle5" jslog="12690;track:click,contextmenu;mutable:true;metadata:WyIwYWhVS0V3aVJyWTJpOFBtR0F4Vk82Y2tESGRJUEE2WVE4QmNJQlNnQSIsbnVsbCwxXQ=="></a>, <a aria-label="Cedis Walmart ecommerce" class="hfpxzc" href="https://www.google.com.mx/maps/place/Cedis+Walmart+ecommerce/data=!4m7!3m6!1s0x842f49df9293e1ff:0xc5cc3c098d25ee6b!8m2!3d20.4796628!4d-103.2616698!16s%2Fg%2F11rcndfsth!19sChIJ_-GTkt9JL4QRa-4ljQk8zMU?authuser=0&amp;hl=es-419&amp;rclk=1" jsaction="pane.wfvdle8;focus:pane.wfvdle8;blur:pane.wfvdle8;auxclick:pane.wfvdle8;keydown:pane.wfvdle8;clickmod:pane.wfvdle8" jslog="12690;tra

In [16]:
info=sopa("a")
nombres=[link.get("aria-label") for link in info]
links=[link.get("href") for link in info]
links

['https://www.google.com.mx/maps/place/Cedis+eCommerce+Walmart/data=!4m7!3m6!1s0x85d2214c2617819f:0x2ac0d663c0d44ef6!8m2!3d19.7467676!4d-99.204067!16s%2Fg%2F11gtynlbtp!19sChIJn4EXJkwh0oUR9k7UwGPWwCo?authuser=0&hl=es-419&rclk=1',
 'https://www.google.com.mx/maps/place/Cedis+Walmart+ecommerce/data=!4m7!3m6!1s0x842f49df9293e1ff:0xc5cc3c098d25ee6b!8m2!3d20.4796628!4d-103.2616698!16s%2Fg%2F11rcndfsth!19sChIJ_-GTkt9JL4QRa-4ljQk8zMU?authuser=0&hl=es-419&rclk=1',
 'https://www.google.com.mx/maps/place/Walmart+Cedis+Perecederos/data=!4m7!3m6!1s0x842f535091f4d66d:0x55538effa5e3fd6a!8m2!3d20.5675263!4d-103.3682508!16s%2Fg%2F11fgh_b4zt!19sChIJbdb0kVBTL4QRav3jpf-OU1U?authuser=0&hl=es-419&rclk=1',
 'https://www.google.com.mx/maps/place/CEDIS+walmart+PERECEDEROS/data=!4m7!3m6!1s0x85ec29dc2da4016d:0xa905ce7f3e5ceaf4!8m2!3d17.9862983!4d-93.193946!16s%2Fg%2F11mtd9jzbj!19sChIJbQGkLdwp7IUR9OpcPn_OBak?authuser=0&hl=es-419&rclk=1',
 'https://www.google.com.mx/maps/place/Walmart+eCommerce+MTY/data=!4m7!3m6!1

In [17]:
coordinates=[]


for url in links:  
    matches = re.findall(r'!3d([-\d.]+)!4d([-\d.]+)', url)
    if matches:
        latitude, longitude = matches[0]  
        coordinates.append((latitude, longitude))


In [19]:
#Pegar nombres y coordenadas en un dataframe
walmart_df=pd.DataFrame({"nombre": nombres, "coordenadas": coordinates})
walmart_df["tienda"]="Walmart"
#Split coordenadas
walmart_df["lat"]=walmart_df["coordenadas"].apply(lambda x: x[0])
walmart_df["lon"]=walmart_df["coordenadas"].apply(lambda x: x[1])
#Eliminar coordenadas
walmart_df.drop("coordenadas", axis=1, inplace=True)
#Minusculas en nombres
walmart_df["nombre"]=walmart_df["nombre"].str.lower()
#Eliminar duplicados en nombres
walmart_df.drop_duplicates(subset="nombre", inplace=True)
walmart_df

,nombre,tienda,lat,lon
0,cedis ecommerce walmart,Walmart,19.7467676,-99.204067
1,cedis walmart ecommerce,Walmart,20.4796628,-103.2616698
2,walmart cedis perecederos,Walmart,20.5675263,-103.3682508
3,cedis walmart perecederos,Walmart,17.9862983,-93.193946
4,walmart ecommerce mty,Walmart,25.7950038,-100.282875
5,walmart cedis perecederos vsa,Walmart,17.9836486,-93.1960445
6,centro de distribucion walmart,Walmart,17.9814468,-93.1635841
7,cedis villahermosa perecederos,Walmart,17.9863452,-93.1955754
8,cedis walmart la luz,Walmart,19.695354,-99.200388
9,cedis walmart refrigerados,Walmart,25.8846732,-100.2346366


#### Geocodear

In [20]:
#Pegar los dos dataframes
df=pd.concat([soriana_df, chedraui_df], ignore_index=True)
df

,nombre,direccion,tipo,tienda
0,CDF Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Fresco,Soriana
1,CDF Cuautitlán,"Autopista México-Querétaro km 36.8,C.P. 54700 ...",Fresco,Soriana
2,CDF Villahermosa,"Carretera Villahermosa-Cárdenas Km 164.1,Ranch...",Fresco,Soriana
3,CDF Guadalajara,"Periférico Sur Manuel Gómez Morín No. 5890,Col...",Fresco,Soriana
4,CDF Tijuana,Prolongación Club de Leones 10535.Fraccionamie...,Fresco,Soriana
5,CDF Hermosillo,Av. Chetumal entre Guatemala y Belice No. 1121...,Fresco,Soriana
6,CDS Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Seco,Soriana
7,CDS Tultitlán,"Km. 14.5 Carretera Puente de Vigas Cuautitlán,...",Seco,Soriana
8,CDS Querétaro,"Autopista México-Querétaro Km 186 S/N, Col Ent...",Seco,Soriana
9,CDS Guadalajara,"Periférico Sur Manuel Gómez Morín #5890, Col. ...",Seco,Soriana


In [30]:
#Credenciales de Google Maps API desde txt. Se debe crear un archivo de texto con las credenciales. Esto es personal.
with open("C:/Users/maestrokarin/Documents/credencialesgmaps.txt", "r") as file:
    api_key=file.read().strip()

gmaps=googlemaps.Client(key=api_key)

In [31]:
#Geocodificar las direcciones y ponerlo en la columna de coordenadas
df["coordenadas"]=df["direccion"].apply(lambda x: gmaps.geocode(x)[0]["geometry"]["location"])
#Crear dos columnas con las coordenadas
df["lat"]=df["coordenadas"].apply(lambda x: x["lat"])
df["lon"]=df["coordenadas"].apply(lambda x: x["lng"])
#Dropear la columna de coordenadas
df=df.drop(columns=["coordenadas"])
df

,nombre,direccion,tipo,tienda,lat,lon
0,CDF Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Fresco,Soriana,25.897103,-100.262871
1,CDF Cuautitlán,"Autopista México-Querétaro km 36.8,C.P. 54700 ...",Fresco,Soriana,19.720678,-99.208614
2,CDF Villahermosa,"Carretera Villahermosa-Cárdenas Km 164.1,Ranch...",Fresco,Soriana,17.974040,-93.130237
3,CDF Guadalajara,"Periférico Sur Manuel Gómez Morín No. 5890,Col...",Fresco,Soriana,20.578165,-103.349850
4,CDF Tijuana,Prolongación Club de Leones 10535.Fraccionamie...,Fresco,Soriana,32.442181,-117.004557
5,CDF Hermosillo,Av. Chetumal entre Guatemala y Belice No. 1121...,Fresco,Soriana,29.107349,-111.002822
6,CDS Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Seco,Soriana,25.897103,-100.262871
7,CDS Tultitlán,"Km. 14.5 Carretera Puente de Vigas Cuautitlán,...",Seco,Soriana,19.512170,-99.214350
8,CDS Querétaro,"Autopista México-Querétaro Km 186 S/N, Col Ent...",Seco,Soriana,20.529097,-100.175101
9,CDS Guadalajara,"Periférico Sur Manuel Gómez Morín #5890, Col. ...",Seco,Soriana,20.578165,-103.349850


In [32]:
#Sustituir lat y long del registro 6 por 29.115216575023382, -111.00023263067902
df.loc[5, "lat"]=29.115216575023382
df.loc[5, "lon"]=-111.00023263067902

In [33]:
#Pegar waalmart y df
df=pd.concat([df, walmart_df], ignore_index=True)
df

,nombre,direccion,tipo,tienda,lat,lon
0,CDF Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Fresco,Soriana,25.897103,-100.262871
1,CDF Cuautitlán,"Autopista México-Querétaro km 36.8,C.P. 54700 ...",Fresco,Soriana,19.720678,-99.208614
2,CDF Villahermosa,"Carretera Villahermosa-Cárdenas Km 164.1,Ranch...",Fresco,Soriana,17.97404,-93.130237
3,CDF Guadalajara,"Periférico Sur Manuel Gómez Morín No. 5890,Col...",Fresco,Soriana,20.578165,-103.34985
4,CDF Tijuana,Prolongación Club de Leones 10535.Fraccionamie...,Fresco,Soriana,32.442181,-117.004557
5,CDF Hermosillo,Av. Chetumal entre Guatemala y Belice No. 1121...,Fresco,Soriana,29.115217,-111.000233
6,CDS Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Seco,Soriana,25.897103,-100.262871
7,CDS Tultitlán,"Km. 14.5 Carretera Puente de Vigas Cuautitlán,...",Seco,Soriana,19.51217,-99.21435
8,CDS Querétaro,"Autopista México-Querétaro Km 186 S/N, Col Ent...",Seco,Soriana,20.529097,-100.175101
9,CDS Guadalajara,"Periférico Sur Manuel Gómez Morín #5890, Col. ...",Seco,Soriana,20.578165,-103.34985


In [35]:
mapa = folium.Map(location=[19.432608, -99.133209], zoom_start=6, tiles="http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}", attr="Google Satellite")


def get_marker_color(tienda_value):
    colors = {
        'Soriana': 'green',
        'Chedraui': 'orange',
        'Walmart': 'blue'
    }
    return colors.get(tienda_value, 'gray')  


for _, row in df.iterrows():
    # Tooltip
    tooltip_text = f"<b>Nombre:</b> {row['nombre']}<br><b>Dirección:</b> {row['direccion']}<br><b>Tipo:</b> {row['tipo']}<br><b>Tienda:</b> {row['tienda']}"
    # marker
    folium.Marker(
        [row["lat"], row["lon"]],
        tooltip=tooltip_text,
        size=15,
        icon=folium.Icon(color=get_marker_color(row['tienda']))
    ).add_to(mapa)




template = """
{% macro html(this, kwargs) %}
<div style="position: fixed; 
     top: 50px; left: 50px; width: 150px; height: auto; 
     border:2px solid grey; z-index:9998; font-size:14px;
     background-color:white; opacity: 0.8; padding: 5px; margin-bottom: 20px;">
    &nbsp; <b>Tiendas</b> <br>
    &nbsp; Walmart &nbsp; <i class="fa fa-circle" style="color:lightblue;"></i><br>
    &nbsp; Soriana &nbsp; <i class="fa fa-circle" style="color:green;"></i><br>
    &nbsp; Chedraui &nbsp; <i class="fa fa-circle" style="color:orange;"></i><br>
</div>
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 900px; 
     z-index:9997; font-size:12px; 
     color: black;
     background-color:white; opacity: 0.8; padding: 5px; margin-top: 20px;">
    <strong>Fuentes:</strong> <br>
    <a href="https://www.organizacionsoriana.com/centros_de_distribucion.html" target="_blank">Soriana: www.organizacionsoriana.com/centros_de_distribucion.html</a><br>
    <a href="https://www.grupochedraui.com.mx/en/division_logistica/index.html" target="_blank">Chedraui: www.grupochedraui.com.mx/en/division_logistica/index.html</a><br>
    <a href="https://www.google.com.mx/maps/search/cedis+perecederos+walmart+ecommerce/@21.7065157,-103.5592671,6z/data=!3m1!4b1?entry=ttu" target="_blank">Walmart: https://www.google.com.mx/maps/search/cedis+perecederos+walmart+ecommerce/@21.7065157,-103.5592671,6z/data=!3m1!4b1?entry=ttu</a><br>
</div>
{% endmacro %}
"""


macro = MacroElement()
macro._template = Template(template)
macro.urls = url


mapa.get_root().add_child(macro)
folium.LayerControl().add_to(mapa)
# mapa
mapa

In [36]:
#Salvar mapa
mapa.save("cedis.html")
